In [454]:
import os
import re
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
stopwords = open("listfile.txt").read().splitlines()
def loadData(loc):
        BASEDIR=os.path.dirname(os.path.realpath('__file__'))
        df=pd.read_csv(BASEDIR+loc,names=["headline","news"])
        return df


In [455]:
df=loadData("/dataset/onlineKhabar.csv")
REPLACE_BY_SPACE_RE = re.compile('[?!/(){}\[\]\|@,;\'\']')

In [456]:
df['news'][0]

"['२० जेठ, काठमाडौं । दूरसञ्चार सेवा प्रदायक कम्पनी नेपाल टेलिकमले हालै रसुवा, ताप्लेजुङ, डोल्पा र मुस्ताङका विभिन्न दूर्गम र विकट भेगहरुमा मोवाइल सेवा शुभारम्भ गरेको छ । हालसम्म मोबाइल तथा टेलिफोन जस्ता सञ्चार सेवाबाट वञ्चित भौगोलिक रुपले अति विकट यी स्थानहरुमा हालै कम्पनीले सञ्चार सेवा पुर्\\u200dयाउन सफल भएको हो ।', 'कम्पनीले ४ हजार २९० मिटर उचाईमा अवस्थित रसुवाको पाङसाङ डाँडामा बीटीएस राखेर सेवा प्रदान गरेको छ । यसबाट लाङटाङ ट्रेकिङ रुट, गोसाइकुन्ड ट्रेकिङ रुटको केही भाग, ठूलो स्याफ्रु, भिम्सा, सिंगवा, नागथली, ग्यात्लाङ, शेर्पा गाउँ, लामा होटल, घोडा तबेला, मुन्टु, सिङडुङ, खाङजिम आदि स्थानमा मोवाइल सेवा उपलब्ध हुनेछ । यसबाट सो क्षेत्रका ५ सय घरधुरीका करिब २ हजार जनसंख्यालाई सेवा उपलब्ध हुनेछ ।', 'रसुवाकै ३ हजार २४२ मिटर उचाईमा अवस्थित लाङटाङ पदमार्गमा पनि अर्को बीटीएस टावर स्थापना गरिएको छ । यसबाट लाङटाङ उपत्यकाका २ सय घरधुरीका करिब ७ सय जनसंख्याले सञ्चार सेवा उपभोग गर्न पाउने छन् ।ताप्लेजुङको विकट स्थान घुन्सामा पनि सेवा उपलब्ध गराइएको छ । ३ हजार ८२५ मिटर उचाईमा अवस्थित टावरमार्फत 

In [457]:
def removeCharacs(text):
    text=re.sub(REPLACE_BY_SPACE_RE,' ',text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    return text

In [458]:
flatten = lambda x : ''.join(x.split('\','))

In [459]:
df['news'] = df['news'].map(flatten).map(removeCharacs)
df['headline'] = df['headline'].map(flatten).map(removeCharacs)


In [460]:
stems =['कै','मै','को','ले','बाट','का','हरु','हरुसँग','सँग','लाई','हरू','हरूसँग','हरू','पटक']
allwords = open("dict.txt").read().splitlines()

In [461]:
def stem_text_cat2(text):
    return re.sub('(मा|को|ले|बाट|का|हरु|हरुसँग|सँग|लाई|हरू|हरूसँग|हरू|पटक)','',text)

In [462]:
suffixes = ('ी' , 'ीया' ,  'ीय' ,'े' , 'नु' , 'दै' , 'ेको' , 'ै')
def stem_text_cat1(word):
    if word.endswith(suffixes):
        wordlist =  [word.rstrip(x) for x in suffixes]
        for i in wordlist:
            if i!= word:
                return i
                    
        
    return word
    

In [463]:
def stemming(text):
    text = ' '.join([stem_text_cat1(text) for word in word_tokenize(text)])
    return ' '.join([stem_text_cat2(a) for a in word_tokenize(text)])
    

In [464]:
df['news'] = df['news'].map(stemming)
df['headline'] = df['headline'].map(stemming)
df['fk'] = df['news']

In [465]:
def sent_tokenize(text):
    sentence = [x for x in text.split('।')] 
    return sentence

In [466]:
df['news']=df['news'].map(sent_tokenize)

In [467]:
df['original'] = df['news']

In [468]:
def check(x):
    try:
        if isinstance(float(x), float):
            return False
    except:
        return True


In [469]:
# function to remove stopwords
def remove_stopword(sen):
    sen_new = " ".join([i for i in sen if i not in stopwords and check(i)])
    return sen_new

def remove_stopword_wnum(sen):
    sen_new = " ".join([i for i in sen if i not in stopwords])
    return sen_new

In [470]:
remove_stopwords = lambda sent : [remove_stopword(r.split()) for r in sent]
remove_stopwords_wnum = lambda sent : [remove_stopword_wnum(r.split()) for r in sent]
df['news']=df['news'].map(remove_stopwords)
df['news-w-num'] = df['news'].map(remove_stopwords_wnum)

In [471]:
df['news'][0]


['जेठ ठडौं',
 'दूरसञ्चार सेवा प्रदायक कम्पनी नेपाल टेलिकम हालै रसुवा ताप्जुङ डोल्पा मुस्ताङ विभिन्न दूर्गम विकट भेग मोवाइल सेवा शुभारम्भ',
 'हालसम्म मोबाइल टेलिफोन सञ्चार सेवा वञ्चित भौगोलिक रुप अति विकट स्थान हालै कम्पनी सञ्चार सेवा पुर्\\u200dयाउन सफल',
 'कम्पनी मिटर उचाई अवस्थित रसुवा पाङसाङ डाँडा बीटीएस राखेर सेवा प्रदान',
 'लाङटाङ ट्रेकिङ रुट गोसाइकुन्ड ट्रेकिङ रुट भाग स्याफ्रु भिम्सा सिंगवा नागथली ग्यात्लाङ शेर्पा गाउँ ला होटल घोडा तबेला मुन्टु सिङडुङ खाङजिम स्थान मोवाइल सेवा उपलब्ध',
 'क्षेत्र घरधुरी करिब जनसंख्या सेवा उपलब्ध',
 'रसुवाकै मिटर उचाई अवस्थित लाङटाङ पदर्ग अर् बीटीएस टावर स्थापना गरिए',
 'लाङटाङ उपत्य घरधुरी करिब जनसंख्या सञ्चार सेवा उपभोग पाउने',
 'ताप्जुङ विकट स्थान घुन्सा सेवा उपलब्ध गराइए',
 'मिटर उचाई अवस्थित टावरर्फत घुन्सा थामे ज्याल्पा कन्चनजंघा बेसक्याम्प स्थान सेवा उपलब्ध',
 'घरधुरी करिब जनसंख्या लाभान्वित हुनेछन्',
 'डोल्पा मुट शे फोक्सुन्डो हालै बीटीएस सञ्चालन',
 'मिटर उचाई मुट अवस्थित टावरर्फत छार् तान्सुङ गाउँपालि वडा इके गाउँपालि वडा तान्छिन खानीगाउँ स

# Sentence Extraction Using PageRank

In [472]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf(corpus):
    vectorizer = TfidfVectorizer(min_df=2,max_df=5)
    try:
        X = vectorizer.fit_transform(corpus)
        return X
    except :
        return

In [473]:
len(df)

22

In [474]:
df['news_vector']=df['news'].map(tfidf)
from sklearn.metrics.pairwise import cosine_similarity

In [475]:
def similarity(sentences):
    try:
        sim_mat = np.zeros([sentences.shape[0],sentences.shape[0]])
        for i in range(sentences.shape[0]):
              for j in range(sentences.shape[0]):
                    if i != j:
                          sim_mat[i][j] = cosine_similarity(sentences[i], sentences[j])[0,0]

        return sim_mat
    except:
        pass

In [476]:
df['similarity']=df['news_vector'].map(similarity)

In [477]:
import networkx as nx
ranks=[]
for i in range(len(df)):
    try:
        nx_graph = nx.from_numpy_array(df['similarity'][i])
        scores = nx.pagerank(nx_graph)
        ranked_sentences = sorted(((scores[k],s) for k,s in enumerate(df['original'][i])), reverse=True)
        ranks.append((ranked_sentences[0:3]))
    except:
        pass

In [478]:
summaries = pd.DataFrame(ranks,columns = ['Summary1','Summary2','Summary3'])

In [479]:
summaries['Summary3']

Series([], Name: Summary3, dtype: object)

In [480]:
df['headline'][0]

'दुर्गम सेवा फैलाउँदै टेलिकम दुर्गम सेवा फैलाउँदै टेलिकम दुर्गम सेवा फैलाउँदै टेलिकम दुर्गम सेवा फैलाउँदै टेलिकम'

# Keyword Extraction

In [481]:
# function to remove stopwords
def remove_stopword(sen):
    sen_new = " ".join([i for i in sen if i not in stopwords and check(i)])
    return sen_new
remove_stopwords = lambda sent : [remove_stopword(r.split()) for r in sent]

In [482]:
def remove_stopwords_w_num(sen):
    sen_new = " ".join([i for i in sen if i not in stopwords])
    return sen_new
remove_stopwords_w_num = lambda sent : [remove_stopword(r.split()) for r in sent]

In [483]:
df['news-ws'] = df['news'].map(remove_stopwords)
df['news-w-num'] = df['news-w-num'].map(remove_stopwords_w_num)

In [484]:
from nltk import word_tokenize

In [485]:
vocab = set([x for x in ' '.join(df['news-ws'][19]).split(' ') ])
vocab = list(vocab)

In [486]:
def checkMe(x):
    try:
        if isinstance(float(x), float):
            return True
    except:
        return False
def checkNum(x):
    try:
        if isinstance(float(x), float):
            return 
    except:
        return x

def findCandidates(text):
    clusters=[]
    word1=[]
    for word in word_tokenize(text):
        if (word in stopwords) or checkMe(word):
            clusters.append(word1)
            word1 =[]
            continue
        else:
            word1.append(word)
    return [' '.join(a).replace('।','') for a in clusters if len(a)>1]
df['kW-candidates'] = df['fk'].map(findCandidates)

In [487]:
from collections import OrderedDict

In [508]:
def find_scores(text):
    document = [checkNum((word_tokenize(x))) for x in text]
    vocab = set([x for x in ' '.join(text).split(' ') ])
    vocab = list(vocab)
    names = vocab
    occurrences = OrderedDict((name, OrderedDict((name, 0) for name in names)) for name in names)

# Find the co-occurrences:
    for l in document:
        for i in range(len(l)):
            for item in l[:i] + l[i + 1:]:
                occurrences[l[i]][item] += 1
                
                
    rows = ['deg','freq','deg/freq']
    word_scores = OrderedDict((name, OrderedDict((r, 0) for r in rows)) for name in names)
    
    
    for key in word_scores.keys():
        word_scores[key]['deg'] = sum(occurrences[key].values())
        word_scores[key]['freq'] = len([x for x in occurrences[key].values() if x!=0])
        if word_scores[key]['freq']!=0:
            word_scores[key]['deg/freq'] = word_scores[key]['deg']/word_scores[key]['freq']
        else:
            word_scores[key]['deg/freq']= 0
            
            
    scoredCandidates = []
    for sent in range(len(text)):
        sumis=0
        for word in word_tokenize(text[sent]):
            sumis += word_scores[word]['deg/freq']
        scoredCandidates.append((text[sent],sumis/len(word_tokenize(text[sent]))))
        
    return sorted(list(set(scoredCandidates)), key = lambda x : x[1],reverse=True)


In [ ]:
df['scored'] = df['kW-candidates'].map(find_scores)

In [ ]:
df['scored'][0]

In [ ]:
def update(text):
    keyword = []
    count = 0
    for i in text:
        if count ==0:
            keyword.append(i[0])
        
        else:
            maxlen = max(len(i[0]),len(keyword[-1]))
            if (nltk.edit_distance(i[0],keyword[-1])/maxlen)<=0.6:
                pass
            else:
            
                keyword.append(i[0])
            
        count+=1
        
    return keyword
        

In [ ]:
df['n-scored'] = df['scored'].map(update)

In [ ]:
keywords = lambda text : ' '.join(set([x for a in text[:5] for x in a.split(' ')]))

In [ ]:
df['keywords'] = df['n-scored'].map(keywords)

In [ ]:
df['keywords'][0]

# Similarity Between Top Sentences and Keywords

In [ ]:
from difflib import SequenceMatcher

In [ ]:
summary_sentence = [] 
for i in range(len(df)):
    ratio=[]
    text1 = df['keywords'][i]
    for j in range(3):
        text = summaries['Summary'+str(j+1)][i][1]
        ratio.append(SequenceMatcher(None, text, text1).ratio())
        
    index = ratio.index(max(ratio))
    
    summary_sentence.append(summaries['Summary'+str(index+1)][i])
    
    
    #print(index,summary_sentence)

In [ ]:
print(summary_sentence[1]) 

# Find Accuracy with ROGUE Measure

In [ ]:
#ROGUE-1 Measure


In [ ]:
def find_recall(x,y):
    '''
    x: generated summary
    y: reference summary
    '''
    similarity = set(word_tokenize(x)).intersection(set(word_tokenize(y)))
    return len(similarity)/len(set(word_tokenize(y)))

In [ ]:
def find_precision(x,y):
    '''
    x: generated summary
    y: reference summary
    '''
    similarity = set(word_tokenize(x)).intersection(set(word_tokenize(y)))
    return len(similarity)/len(set(word_tokenize(x)))

In [ ]:
def find_rogue_accuracy( x , y ):
    print("ROGUE Precision is, ",find_precision(x,y))
    print("ROGUE Recall is, ",find_recall(x,y))
    

In [ ]:
def stem_r_stopwords(text):
    return ' '.join([x for x in word_tokenize(text) if x not in stopwords])

In [ ]:
def find_acc(i):
    find_rogue_accuracy(stem_r_stopwords(summary_sentence[i][1]),stem_r_stopwords(df['headline'][i]))
    print(stem_r_stopwords(summary_sentence[i][1]),'***',stem_r_stopwords(df['headline'][i]))

In [ ]:
find_acc(19)